<a href="https://colab.research.google.com/github/gkoorsen/Automatic_multi_docking/blob/main/Docking_template_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.2 MB/s eta 0:00:00


In [ ]:
!pip install python-dotenv

In [ ]:
from Bio.PDB import PDBParser
import Bio.PDB as PDB
from google.colab import files
import pandas as pd
import os
import requests
from Bio.Blast import NCBIWWW, NCBIXML
import time
from dotenv import load_dotenv





In [ ]:

def download_pdb_file(pdb_id: str) -> str:

    PDB_DIR ="/tmp/pdb/"
    os.makedirs(PDB_DIR, exist_ok=True)

    # url or pdb_id
    if pdb_id.startswith('http'):
        url = pdb_id
        filename = url.split('/')[-1]
    elif pdb_id.endswith(".pdb"):
        return pdb_id
    else:
        if pdb_id.startswith("AF"):
            url = f'https://alphafold.ebi.ac.uk/files/{pdb_id}-model_v3.pdb'
        else:
            url = f'http://files.rcsb.org/view/{pdb_id}.pdb'
        filename = f'{pdb_id}.pdb'

    cache_path = os.path.join(PDB_DIR, filename)
    if os.path.exists(cache_path):
        return cache_path

    pdb_req = requests.get(url)
    pdb_req.raise_for_status()
    open(cache_path, 'w').write(pdb_req.text)
    return cache_path

def find_ligands(pdb_file):

    ligand_counts = {}
    ligand_names = {}

    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith("HETNAM"):
                ligand_code = line[11:14].strip()
                ligand_name = line[15:].strip()
                ligand_names[ligand_code] = ligand_name
                print(ligand_code,ligand_name)
            elif line.startswith("HET "):
                het_ligand = line[7:10].strip()
                chain = line[12].strip()
                if het_ligand not in ligand_counts:
                    ligand_counts[het_ligand] = {'chain' : [], 'counts' : []}
                if chain not in ligand_counts[het_ligand]['chain']:
                    ligand_counts[het_ligand]['chain'].append(chain)
                    ligand_counts[het_ligand]['counts'].append(1)
                else:
                    position = ligand_counts[het_ligand]['chain'].index(chain)
                    ligand_counts[het_ligand]['counts'][position] += 1

    return ligand_names, ligand_counts


def analyse_ligands(pdb_files):
    pdbs = []
    ligs = []
    names = []
    chains = []
    counts = []

    for pdb_file in pdb_files:
        print(f"Processing file: {pdb_file}")
        ligand_dict, ligand_count_dict = find_ligands(pdb_file)
        print(f"Ligands found: {list(ligand_dict.items())}")
        for ligand, chains_and_counts in ligand_count_dict.items():
            if ligand == ' ':
                continue
            for chain, count in zip(chains_and_counts['chain'], chains_and_counts['counts']):
                pdbs.append(pdb_file[pdb_file.find('/pdb/')+5:pdb_file.find('.pdb')])
                ligs.append(ligand)
                names.append(ligand_dict.get(ligand, "Unknown"))
                chains.append(chain)
                counts.append(count)
    data = pd.DataFrame({
        'pdb': pdbs,
        'ligand': ligs,
        'name': names,
        'chain': chains,
        'count': counts
    })
    return data



def parse_pdb_line(line):
    if line.startswith("HETNAM"):
        ligand_code = line[11:14].strip()
        ligand_name = line[15:].strip()
        if not ligand_code:  # Ignore lines where ligand_code is empty
            return (None, None, None, None)
        return (ligand_code, ligand_name, None, None)
    elif line.startswith("HET "):
        het_ligand = line[7:10].strip()
        chain = line[21].strip()  # Remove any potential space in chain as well
        if not het_ligand or not chain:  # Ignore lines where het_ligand or chain is empty
            return (None, None, None, None)
        return (het_ligand, None, chain, None)
    return (None, None, None, None)


def find_ligands(pdb_file):

    ligand_counts = {}
    ligand_names = {}

    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith("HETNAM"):
                ligand_code = line[11:14].strip()
                ligand_name = line[15:].strip()
                ligand_names[ligand_code] = ligand_name
            elif line.startswith("HET "):
                het_ligand = line[7:10].strip()
                chain = line[12:13].strip()  # Adjusted here
                if het_ligand not in ligand_counts:
                    ligand_counts[het_ligand] = {'chain' : [], 'counts' : []}
                if chain not in ligand_counts[het_ligand]['chain']:
                    ligand_counts[het_ligand]['chain'].append(chain)
                    ligand_counts[het_ligand]['counts'].append(1)
                else:
                    position = ligand_counts[het_ligand]['chain'].index(chain)
                    ligand_counts[het_ligand]['counts'][position] += 1

    return ligand_names, ligand_counts



def find_similar_structure_with_ligand(pdb_id, seq_identity_cutoff):
    print(f'Retrieving the protein sequence for {pdb_id[pdb_id.find("pdb/")+4:]}...')
    ppb = PDB.PPBuilder()
    pdb_id_only = pdb_id.split('/')[-1]  # get only the last part after '/'
    structure = PDB.PDBParser().get_structure(pdb_id_only, pdb_id)
    for pp in ppb.build_peptides(structure):
        sequence = pp.get_sequence()
    print(f'Performing a BLAST search using this sequence...')
    result_handle = NCBIWWW.qblast("blastp", "pdb", sequence)
    blast_record = NCBIXML.read(result_handle)
    print('Looking at the alignments of each similar structure...')
    similar_structures_with_ligands = []
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            seq_identity = hsp.identities / hsp.align_length
            if seq_identity < seq_identity_cutoff:
                continue
            similar_pdb_id = alignment.hit_id.split("|")[1].split("_")[0]
            print('Check if the similar structure has a ligand')
            time.sleep(5)
            pdb_file = download_pdb_file(similar_pdb_id)
            ligand_names, ligand_counts = find_ligands(pdb_file)
            if set(ligand_names.keys())-set(excluded) != set():
                print(f'Structure found: {similar_pdb_id}')
                similar_structures_with_ligands.append(similar_pdb_id)
    return similar_structures_with_ligands





In [ ]:
output_file = 'output.xlsx'

uninteresting_molecules = [
    'TRIS(HYDROXYETHYL)AMINOMETHANE',
    'SULFATE ION',
    'SODIUM ION',
    'CALCIUM ION',
    'CHLORIDE ION',
    'GLYCEROL',
    'ZINC ION',
    '1,2-ETHANEDIOL',
    'DIMETHYL SULFOXIDE',
    'MAGNESIUM ION',
    'PHOSPHATE ION',
    'TETRAFLUOROALUMINATE ION',
    'TRIETHYLENE GLYCOL',
    'DI(HYDROXYETHYL)ETHER',
    'ETHANOL',
    'METHANOL',
    'WATER',
    'PROPANE',
    'BETA-MERCAPTOETHANOL',
    '1,2-PROPANEDIOL',
    'ACETATE ION',
    'FORMATE ION',
    'NITRATE ION',
    'CARBONATE ION',
    'MALONATE ION',
    'CITRATE ION',
    'TARTRATE ION',
    'ASCORBATE ION',
    'GLUTAMATE ION',
    "ADENOSINE-5'-MONOPHOSPHATE",
    "GUANOSINE-5'-MONOPHOSPHATE",
    "CYTIDINE-5'-MONOPHOSPHATE",
    "THYMIDINE-5'-MONOPHOSPHATE",
    "URIDINE-5'-MONOPHOSPHATE",
    "ACETYL GROUP",
    "AMINO GROUP",
    "THIOCYANATE ION",
    "SELENOMETHIONINE",
    "GLYCEROL"
]


In [ ]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name)

In [ ]:
PDB_files = []

for pdb_id in df['PDB IDs'].dropna():
    PDB_files.append(download_pdb_file(pdb_id.strip()))



#Initialize set of all PDBs and a set to store those we've already checked
all_pdbs = set(PDB_files)
checked_pdbs = set()

while True:  # This will keep running until we manually break
    new_pdbs = set()  # Store any new PDBs we find during this iteration
    for pdb in all_pdbs - checked_pdbs:
        # Get the data for this PDB
        pdb_data = analyse_ligands([pdb])
        # Select only the rows corresponding to interesting ligands
        interesting_ligand_data = pdb_data[~pdb_data['name'].isin(uninteresting_molecules)]
        print(f'{pdb} has the following interesting data {interesting_ligand_data}')
        # If there are no interesting ligands, find similar structures
        if interesting_ligand_data.empty:
            try:
                similar_pdb_ids = find_similar_structure_with_ligand(pdb, 0.9)
                for new_pdb_id in similar_pdb_ids:
                    new_pdb_file = download_pdb_file(new_pdb_id)
                    new_pdbs.add(new_pdb_file)
            except Exception as e:
                print(f"Error occurred while finding similar structures for PDB {pdb}: {e}")
    if not new_pdbs:
        break
    # Add the new PDBs to our set of all PDBs
    all_pdbs.update(new_pdbs)
    # Mark the PDBs we've checked during this iteration as checked
    checked_pdbs.update(all_pdbs - checked_pdbs)

# We can now analyze them all together
final_data = analyse_ligands(list(all_pdbs))


# Filter out uninteresting ligands
interesting_ligand_final_data = final_data[~final_data['name'].isin(uninteresting_molecules)]
interesting_ligand_final_data.to_excel(output_file)
files.download(output_file)